# MNLI

## Preparing the Environment

In [ ]:
#Google Colab - Drive Mounting
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#Install missing library keras-nlp
!pip install -q keras-nlp

In [ ]:
#Import the libraries
import tensorflow as tf
import keras_nlp
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import re
import string
import random
import pandas as pd
import csv

## Data Preprocessing and Parameters Initialization

In [ ]:
#Finetuning Parameters
FINETUNING_LEARNING_RATE = 5e-5
FINETUNING_EPOCHS = 3
FINETUNING_BATCH_SIZE = 32
SEQ_LENGTH = 128

In [ ]:
# Download of the vocabulary from BERT: Bert-uncased
vocab_file = keras.utils.get_file(
    origin="https://storage.googleapis.com/tensorflow/keras-nlp/examples/bert/bert_vocab_uncased.txt",
)
#Initialization of the Word Tokenizer, with a given vocabulary and a sequence length
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab_file, sequence_length=SEQ_LENGTH,
)

In [ ]:
#Definition of the classes dictionary
CLASSES = {
    'neutral': 0,
    'entailment': 1,
    'contradiction': 2,
}


current_path = "path_to_MNLI/MNLI"
train_path = os.path.join(current_path, 'train.tsv')
dev_matched_path = os.path.join(current_path, 'dev_matched.tsv')
dev_mismatched_path = os.path.join(current_path, 'dev_mismatched.tsv')

Reference https://github.com/CyberZHG/keras-xlnet/blob/master/demo/GLUE/MNLI/mnli.py

In [ ]:
# Definition of the Class Data using Keras Sequence format
class DataSequence(keras.utils.Sequence):
    '''
    Organize the data into: encoded text/ label 
    Overwrite the Keras.Utils.Sequence Class
    '''
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return (len(self.y) + FINETUNING_BATCH_SIZE - 1) // FINETUNING_BATCH_SIZE

    def __getitem__(self, index):
        s = slice(index * FINETUNING_BATCH_SIZE, (index + 1) * FINETUNING_BATCH_SIZE)
        return [item[s] for item in self.x], self.y[s]


def generate_sequence(path):
    '''
    Given a file path, read the file and organize the data from the file
    Return Keras.utils.Sequence object ready to use for training, with the format: tokens/classes
    '''
    tokens, classes = [], []
    df = pd.read_csv(path, sep='\t', error_bad_lines=False)
    for _, row in df.iterrows():
        text_a, text_b, cls = row['sentence1'], row['sentence2'], row['gold_label']
        if not isinstance(text_a, str) or not isinstance(text_b, str) or cls not in CLASSES:
            continue
        text = (text_a)+ '[SEP]' + (text_b) + '[SEP]'
        encoded = tokenizer(text)
        tokens.append(encoded)
        classes.append(CLASSES[cls])
    tokens, classes = np.array(tokens), np.array(classes)
    return DataSequence([tokens], classes)

In [ ]:
#Load the training data
train_data = generate_sequence(train_path)
dev_matched_data = generate_sequence(dev_matched_path)
dev_mismatched_data = generate_sequence(dev_mismatched_path)

## Load the model and change the head

In [ ]:
# Load the pretrained model and display its structure
model = keras.models.load_model('path_to_the_pretrained_model',compile=False)
model.summary()

In [ ]:
# The model's head is modified for classification

inputs = keras.Input(shape=(SEQ_LENGTH,), dtype=tf.int32)
encoded_tokens = model(inputs)
#HEAD FROM https://keras.io/examples/nlp/text_classification_with_transformer/
x = keras.layers.GlobalAveragePooling1D()(encoded_tokens)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(768, activation="tanh")(x)
x = keras.layers.Dropout(0.1)(x)
outputs = keras.layers.Dense(3, activation="softmax")(x)


finetuning_model = keras.models.Model(inputs=inputs, outputs=outputs)
finetuning_model.summary()

## Model Training

In [ ]:
#Create tensorboard callback
logdir = "path_to_save_execution_information" #+ datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

#Compile model 
finetuning_model.compile(
    optimizer=keras.optimizers.Adam(lr=3e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [ ]:
#Model training
finetuning_model.fit_generator(
    generator=train_data,
    validation_data=dev_matched_data,
    epochs=FINETUNING_EPOCHS,
    callbacks=[tensorboard_callback],
)

In [ ]:
# Add Tokenization layer to the model
inputs = keras.Input(shape=(), dtype=tf.string)
tokens = tokenizer(inputs)
outputs = finetuning_model(tokens)

#Save model
final_model = keras.Model(inputs, outputs)
final_model.save("path_to_save_model")

In [ ]:
#Restore model
restored_model = keras.models.load_model("path_to_save_model", compile=False)

## Testing

In [ ]:
current_path = "path_to_MNLI/MNLI"
test_match_path = os.path.join(current_path, 'test_matched.tsv')
test_mismatched_path = os.path.join(current_path, 'test_mismatched.tsv')


def generate_test(path):
    '''
    Given a file path, read the file and preprocess the data
    Return a numpy array of the encoded test sentences
    '''
    tokens, classes = [], []
    df = pd.read_csv(path, sep='\t',quoting=csv.QUOTE_NONE)
    for _, row in df.iterrows():
        text_a, text_b = row['sentence1'], row['sentence2']
        if not isinstance(text_a, str) or not isinstance(text_b, str):
            text_a = str(text_a)
            text_b = str(text_b)
        text = str(text_a)+ '[SEP]' + str(text_b) + '[SEP]'
        encoded = (text)
        tokens.append(encoded)
    tokens = (tokens)
    return tokens 

#Load the test data
test_m = generate_test(test_match_path)
test_mm = generate_test(test_mismatched_path)

In [ ]:
#Generate predictions for matched test data and mismatched test data 
pred_m = restored_model.predict(test_m, batch_size=128).argmax(axis=-1)
pred_mm = restored_model.predict(test_mm, batch_size=128).argmax(axis=-1)

In [ ]:
#Print the number of results
print(len(pred_m))

In [ ]:
#Format results into dataframe, ready to be uploaded on gluebenchmark.com

REV_CLASSES = {
    0: 'neutral',
    1: 'entailment',
    2: 'contradiction',
}

results_m = []
results_mm = []

for p in pred_m:
  results_m.append(REV_CLASSES[p])

for p in pred_mm:
  results_mm.append(REV_CLASSES[p])

df_m = pd.DataFrame(results_m)
df_mm = pd.DataFrame(results_mm)

df_m.to_csv("MNLI-m.tsv",sep='\t', encoding='utf-8')
df_mm.to_csv("MNLI-mm.tsv",sep='\t', encoding='utf-8')

In [ ]:
#Load Tensorboard
%reload_ext tensorboard
%tensorboard --logdir="path_to_save_execution_information"

In [ ]:
#Code to automatically stop the run time for Google Colab
import time
time.sleep(60)
from google.colab import runtime
runtime.unassign()